# Implementation of "Fair Decisions Despite Imperfect Predictions" using the covariance fairness constraint

In [1]:
import numpy as np
from src.consequential_learning import collect_data, train
from src.feature_map import IdentityFeatureMap

## The parameters used by the the original authors  

In [2]:
training_parameters = {
    'dim_x': 1,
    'dim_s': 1,
    'time_steps':200,
    'batch_size':512,
    'num_iterations': 32,
    'learning_parameters': {
        'learning_rate': 0.5,
        'decay_rate': 0.8,
        'decay_step': 30
    },
    'fairness_rate':0.5,
    'cost_factor':0.55,
    'fraction_protected':0.3
}
training_parameters['dim_theta'] = training_parameters['dim_x'] + training_parameters['dim_s']
training_parameters['feature_map'] = IdentityFeatureMap(training_parameters['dim_theta'])
training_parameters['num_decisions'] = training_parameters['num_iterations'] * training_parameters['batch_size']

## Definition of the fairness function  
The covariance fairness constraint is defined as the covariance between the sensitive attribute and the signed distance from the users feature vectors to the decision boundary: $$Cov_{DI}(s, d_{\theta}) = E[(s - \mu_s)e]$$

In [3]:
def fairness_function(**fairness_kwargs):
    x = fairness_kwargs['x']
    s = fairness_kwargs['s']
    sample_theta = fairness_kwargs['sample_theta']
    policy = fairness_kwargs['policy']
    gradient = fairness_kwargs['gradient']

    ips_weight, phi, log_gradient_denominator = policy.calculate_ips_weights_and_log_gradient(x, s, sample_theta)
    decision = policy(x, s).reshape(-1, 1)

    mu_s = (ips_weight * s).mean(axis=0)
    covariance = (s - mu_s) * decision

    if gradient:
        grad_benefit = ((ips_weight/log_gradient_denominator) * covariance * phi).sum(axis=0) / x.shape[0]
        return grad_benefit
    else:
        benefit = (ips_weight * covariance).sum(axis=0) / x.shape[0]
        return benefit


## Train the model

In [4]:
i = 1
for utility, benefit_delta in train(**training_parameters, fairness_function=fairness_function):
    print("Time step {}: Utility {} \n\t Benefit Delta {}".format(i, utility, benefit_delta))
    i += 1

Fairness penalty: [0.00502812]
Time step 1: Utility -0.021704983142152225
Fairness penalty: [0.01187556]
Time step 2: Utility 0.03989369292069194
Fairness penalty: [0.01383579]
Time step 3: Utility 0.04196568076247986
Fairness penalty: [0.01075765]
Time step 4: Utility 0.04392143452010361
Fairness penalty: [0.01020751]
Time step 5: Utility 0.05591485255878856
Fairness penalty: [0.00801472]
Time step 6: Utility 0.047818266040886265
Fairness penalty: [0.00677428]
Time step 7: Utility 0.05146036878248665
Fairness penalty: [0.00767242]
Time step 8: Utility 0.05332216915084003
Fairness penalty: [0.0129455]
Time step 9: Utility 0.05032895392171851
Fairness penalty: [0.00882702]
Time step 10: Utility 0.050545437785321494
Fairness penalty: [0.01307406]
Time step 11: Utility 0.044051211067691
Fairness penalty: [0.0075396]
Time step 12: Utility 0.053571509783135934
Fairness penalty: [0.00563055]
Time step 13: Utility 0.05936945123589245
Fairness penalty: [0.00451824]
Time step 14: Utility 0.0701